<a href="https://colab.research.google.com/github/Ammulya/Twitter-Sentiment-Analysis/blob/master/Twitter_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [0]:
import os
os.chdir('/drive/My Drive/PROJECTS')

In [0]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk

In [4]:
#Loading the data :
data = pd.read_csv('./datasets/train.csv',encoding="latin-1").loc[0:9999]
data.head(10)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
5,6,0,or i just worry too much?
6,7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,8,0,Sunny Again Work Tomorrow :-| ...
8,9,1,handed in my uniform today . i miss you ...
9,10,1,hmmmm.... i wonder how she my number @-)


In [5]:
# check the number of positive vs. negative tagged sentences
positives = data['Sentiment'][data.Sentiment	 == 0]
negatives = data['Sentiment'][data.Sentiment	 == 1]
print('number of positve tagged sentences is:  {}'.format(len(positives)))
print('number of negative tagged sentences is: {}'.format(len(negatives)))
print('total length of the data is:            {}'.format(data.shape[0]))

number of positve tagged sentences is:  5812
number of negative tagged sentences is: 4188
total length of the data is:            10000


# Text preprocessing:

### 1.Removing special charaters,un neccassary things except alphabets:

In [0]:
#Function to preprocessing theSentiment Text column
def textprocess(text):
    text = text.lower()#convert to lower case
    text = re.sub(r'[^a-z]+',' ',text)#subsitute all values with spaces except alphabets
    text = re.sub(r'\b\w{1,2}\b', '', text)#It will subsitute all the short words with space
    return text

In [7]:
#Applying the Text process function on the data
data['SentimentText']=data['SentimentText'].apply(textprocess)
len(data['SentimentText'])
data.head(10)

,ItemID,Sentiment,SentimentText
0,1,0,sad for apl friend
1,2,0,missed the new moon trailer
2,3,1,omg its already
3,4,0,omgaga sooo gunna cry been this dentist ...
4,5,0,think cheating
5,6,0,just worry too much
6,7,1,juuuuuuuuuuuuuuuuussssst chillin
7,8,0,sunny again work tomorrow tonight
8,9,1,handed uniform today miss you already
9,10,1,hmmmm wonder how she number


### 2.Stemming (or) Lemmatization

In [0]:
#Importing neccessary modules for stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [9]:
#Applying the stemming
data['Sentimentstem'] = data['SentimentText'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split() ]))
data.head(10)


,ItemID,Sentiment,SentimentText,Sentimentstem
0,1,0,sad for apl friend,sad for apl friend
1,2,0,missed the new moon trailer,miss the new moon trailer
2,3,1,omg its already,omg it alreadi
3,4,0,omgaga sooo gunna cry been this dentist ...,omgaga sooo gunna cri been thi dentist sinc wa...
4,5,0,think cheating,think cheat
5,6,0,just worry too much,just worri too much
6,7,1,juuuuuuuuuuuuuuuuussssst chillin,juuuuuuuuuuuuuuuuussssst chillin
7,8,0,sunny again work tomorrow tonight,sunni again work tomorrow tonight
8,9,1,handed uniform today miss you already,hand uniform today miss you alreadi
9,10,1,hmmmm wonder how she number,hmmmm wonder how she number


In [0]:
#Importing neccessary modules for lemmatization
from nltk.stem import wordnet                  #dictionary sought of thing contains corpus of words
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize , pos_tag       #word_tokenize -- will divide the sentences into words
from nltk import pos_tag                       #pos_tag -- for getting the parts of speech of each word

In [0]:
lemma  = WordNetLemmatizer()

In [0]:
#Function for defining  parts of speech for each word
def lemmatizer(pos_tag):
    word,pos = pos_tag
    if pos.startswith('R'):
        pos = 'r'#adverb
    elif pos.startswith('N'):
        pos= 'n'#noun
    
    elif pos.startswith('J'):
        pos= 'a'#adjective
    else:
        pos = 'v'#verb
        
    return lemma.lemmatize(word,pos=pos)


In [0]:
#applying the pos_tag and to get the rootword of each word 
def lemmatization(sent):
    tokens = sent.split()
    pos = pos_tag(tokens)
    return " ".join([lemmatizer(tag) for tag in pos])

In [14]:
#Have to download to get these modules
import nltk
nltk.download('averaged_perceptron_tagger')#for pos_tag

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
#Have to download to get these modules
import nltk
nltk.download('wordnet')#dictionary of words

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
#applying the lemmatization function to data
data['Sentimentlemma'] = data['SentimentText'].apply(lemmatization)
data.tail(10)

,ItemID,Sentiment,SentimentText,Sentimentstem,Sentimentlemma
9990,10002,1,quot came play quot now done lol night ni...,quot came play quot now done lol night night all,quot come play quot now do lol night night all
9991,10003,1,quot can taste you lips and smell you cl...,quot can tast you lip and smell you cloth quot...,quot can taste you lip and smell you clothe qu...
9992,10004,1,quot did not have sexual relations with that...,quot did not have sexual relat with that woman...,quot do not have sexual relation with that wom...
9993,10005,1,quot way shit folks the opposite quot,quot way shit folk the opposit quot,quot way shit folk the opposite quot
9994,10006,0,quot not like plastic grr silly and bad fo...,quot not like plastic grr silli and bad for th...,quot not like plastic grr silly and bad for th...
9995,10007,1,quot don have the strength stay away from ...,quot don have the strength stay away from you ...,quot don have the strength stay away from you ...
9996,10008,0,quot don know what whenever you are near ...,quot don know what whenev you are near quot ka...,quot don know what whenever you be near quot k...
9997,10009,0,quot don wanna see the end quot tomorrow wi...,quot don wanna see the end quot tomorrow will ...,quot don wanna see the end quot tomorrow will ...
9998,10010,0,quot don want anything more than see your ...,quot don want anyth more than see your face wh...,quot don want anything more than see your face...
9999,10011,1,quot eat therefore quot tweet and fill t...,quot eat therefor quot tweet and fill the gap ...,quot eat therefore quot tweet and fill the gap...


### 3.Removing Stop Words

In [17]:
#Have to download to get these modules
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
#importing the library from nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [0]:
#function to remove the stopwords from the data
def stopwords(data):
    data = data.split()
    data_list = [word for word in data if word not in stop]
   #3data_list = [word for word in data if word.isalpha()]
    return " ".join(data_list)
    #return tokens

In [20]:
#applying the stopwords function on lemmatized column
data['Sentimentstop']=data['Sentimentlemma'].apply(stopwords)
data.head(20)

,ItemID,Sentiment,SentimentText,Sentimentstem,Sentimentlemma,Sentimentstop
0,1,0,sad for apl friend,sad for apl friend,sad for apl friend,sad apl friend
1,2,0,missed the new moon trailer,miss the new moon trailer,miss the new moon trailer,miss new moon trailer
2,3,1,omg its already,omg it alreadi,omg its already,omg already
3,4,0,omgaga sooo gunna cry been this dentist ...,omgaga sooo gunna cri been thi dentist sinc wa...,omgaga sooo gunna cry be this dentist since be...,omgaga sooo gunna cry dentist since suposed ge...
4,5,0,think cheating,think cheat,think cheat,think cheat
5,6,0,just worry too much,just worri too much,just worry too much,worry much
6,7,1,juuuuuuuuuuuuuuuuussssst chillin,juuuuuuuuuuuuuuuuussssst chillin,juuuuuuuuuuuuuuuuussssst chillin,juuuuuuuuuuuuuuuuussssst chillin
7,8,0,sunny again work tomorrow tonight,sunni again work tomorrow tonight,sunny again work tomorrow tonight,sunny work tomorrow tonight
8,9,1,handed uniform today miss you already,hand uniform today miss you alreadi,hand uniform today miss you already,hand uniform today miss already
9,10,1,hmmmm wonder how she number,hmmmm wonder how she number,hmmmm wonder how she number,hmmmm wonder number


##  Counting the Number of times each word occured

In [0]:
#spiltting each word in data
data['Sentimentstop']=data['Sentimentstop'].str.split().apply(lambda x: ','.join(list(set(x))))

In [22]:

word_list = []

for i in range(0, len(data['Sentimentstop'])):
    word_list.append(data['Sentimentstop'][i])
    i+=1

print(word_list)


['sad,friend,apl', 'moon,new,trailer,miss', 'already,omg', 'omgaga,since,gunna,cry,put,min,get,dentist,crown,sooo,suposed', 'think,cheat', 'worry,much', 'juuuuuuuuuuuuuuuuussssst,chillin', 'sunny,tomorrow,work,tonight', 'uniform,already,today,miss,hand', 'hmmmm,number,wonder', 'must,think,positive', 'hater,thanks,day,face', 'weekend,far,suck', 'isnt,australia,show', 'win,thats', 'feel,way,right', 'funny,twitter,man,myloc,useless,awhhe,http,completely', 'semisonic,listen,fine,gonna,celebrate,strangely,feel', 'huge,scary,roll,thunder', 'year,happy,grow,shaunamanu,well,gonna,cut,meantime,start,beard', 'sad,iran', 'wompppp,wompp', 'else,one,follow,awesome,cause,see,pretty', 'shut,blog,comp,lose,position,fetal,myspace,level,lay,sad,tweet,write,massive', 'tourny,think,golf,something,pull,rip,head,place,yeah,hospitol', 'please,tell,bore,whats,wrong', 'could,go,life,gig,wish,bother,sit,rest,seriously,spend', 'assignment,dance,like,nooo,shit,right,really,feeeling,sleep,hour,finish,art,want', 'g

In [0]:
text =str(word_list)

In [0]:
text= text.lower()
text = re.sub(r'[^a-z]',' ',text)
token = text.split()

In [25]:
type(token)

list

In [0]:
#import the FreqDist for frequency ditiction
from nltk.probability import FreqDist
f = FreqDist()

In [27]:
#we are going to count the words:
for word in token:
    f[word.lower()]+=1
print(f)

<FreqDist with 14740 samples and 68560 outcomes>


In [28]:
#checking the top most common words
a = f.most_common(100)
a

[('quot', 1025),
 ('get', 833),
 ('http', 640),
 ('day', 527),
 ('amp', 521),
 ('good', 464),
 ('love', 455),
 ('like', 453),
 ('go', 423),
 ('followfriday', 398),
 ('want', 386),
 ('make', 374),
 ('work', 354),
 ('one', 341),
 ('today', 335),
 ('miss', 331),
 ('time', 326),
 ('com', 314),
 ('know', 296),
 ('think', 273),
 ('see', 269),
 ('back', 259),
 ('sad', 253),
 ('feel', 243),
 ('lol', 240),
 ('need', 234),
 ('twitter', 232),
 ('well', 230),
 ('really', 229),
 ('say', 225),
 ('new', 223),
 ('come', 219),
 ('still', 216),
 ('night', 214),
 ('bit', 209),
 ('last', 203),
 ('great', 194),
 ('much', 192),
 ('bad', 192),
 ('follow', 189),
 ('leave', 188),
 ('look', 183),
 ('take', 174),
 ('happy', 173),
 ('home', 169),
 ('hate', 168),
 ('friend', 166),
 ('right', 165),
 ('watch', 158),
 ('people', 158),
 ('win', 157),
 ('guy', 157),
 ('thing', 157),
 ('wish', 155),
 ('tomorrow', 147),
 ('would', 144),
 ('tonight', 143),
 ('sleep', 143),
 ('tweet', 141),
 ('song', 141),
 ('gonna', 139),

#  Feature Extraction/Engineering

###  Bag of words/Counter Vectorizer:

In [0]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [0]:
cv = CountVectorizer()

In [0]:
#X = cv.fit_transform(data['Sentimentstop']).toarray()

In [0]:
#print(cv.get_feature_names())

### TFIDF---Term Frequency -Inverse Document Frequeny

In [0]:
tf = TfidfVectorizer()

In [0]:
X = tf.fit_transform(data['Sentimentstop']).toarray()

In [35]:
#Our independent variable---data['Sentimentstop']
X.shape #Here  we got about 14738 features

(10000, 14738)

In [0]:
#Our dependent/Target Variable
y = data['Sentiment']	

# Applying  a Model:

In [0]:
#importing the models using both NavieBayes and Logistic Regression models
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression

In [0]:
#splitting the data
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

In [0]:
#defining the model
model_nb = MultinomialNB()
model_log = LogisticRegression(C= 10.0)

In [40]:
#fitting the model
model_nb.fit(x_train,y_train)
model_log.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
#importing to get the confusion matrix
!pip install pandas-ml


In [0]:
#predicting the results
y_pred_log = model_nb.predict(x_test)
y_pred_nb=model_log.predict(x_test)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
#confusion matrix for both the models
cm_nb= confusion_matrix(y_test,y_pred_log)
cm_log = confusion_matrix(y_test,y_pred_nb)

In [0]:
##classification report for both the models
cm_nb1=  classification_report(y_test,y_pred_log)
cm_log1 =  classification_report(y_test,y_pred_nb)

In [46]:
print(cm_log1)
#Here we got an accuracy of 75% for logistic regression

              precision    recall  f1-score   support

           0       0.77      0.82      0.79      1753
           1       0.72      0.65      0.68      1247

    accuracy                           0.75      3000
   macro avg       0.74      0.73      0.74      3000
weighted avg       0.75      0.75      0.75      3000



In [47]:
print(cm_nb1)
#Here we got an accuracy of 73% for Navie Bayes

              precision    recall  f1-score   support

           0       0.70      0.95      0.80      1753
           1       0.85      0.43      0.57      1247

    accuracy                           0.73      3000
   macro avg       0.78      0.69      0.69      3000
weighted avg       0.76      0.73      0.71      3000



### Save the model

In [48]:
#importing the saving model tools joblib,pickle
import pickle
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [49]:
joblib.dump(model_log,'new.pkl')#creating the new pickle file

['new.pkl']

In [0]:
log_joblib = joblib.load('new.pkl') #loading for futher use

In [51]:
log_joblib.score(x_test,y_test)#just test for test data too check for proper working/not

0.7483333333333333

In [52]:
log_joblib.predict(x_test)

array([0, 0, 1, ..., 0, 0, 1])

### Testing the model with New raw data


In [0]:
import re

In [0]:
test = 'I am so sad today '#new twitt to check for our model performance


In [0]:
new = 'I love these food '

In [0]:
#preprocessing the new data
t = textprocess(test)

In [0]:
n = textprocess(new)

In [0]:
#Feature extracting from it 
x_sample = tf.transform([t]).toarray()

In [0]:
x_new = tf.transform([n]).toarray()

In [67]:
x_new.shape

(1, 14738)

In [57]:
x_sample.shape

(1, 14738)

In [58]:
log_joblib.predict(x_sample) #predicting the new data with the model save

array([0])

In [68]:
log_joblib.predict(x_new) 

array([1])

In [59]:
model_nb.predict_proba(x_sample)

array([[0.95942483, 0.04057517]])

### Checking for the cosine similarity

In [0]:
#To check how  our model data match the new data
from sklearn.metrics.pairwise import cosine_similarity 

In [61]:
cosine_similarity(X,x_sample).argmax()#It will give you the in which row the maximum matching is done

2076

In [62]:
cosine_similarity(X,x_sample).max()#checking for maximum matching possibility

0.7285802467317896

In [63]:
cosine_similarity(X,x_sample)#comparing the new data with the previous one
#Here the new data matches old data only 29% it seems

array([[0.28946347],
       [0.        ],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])